In [1]:
import lib.io_ as io
import pandas as pd

In [2]:
%%timeit
df = io.load_path_pandas('/data/mlhd-zstd/d3/d38db397-d7b6-47fd-b57b-d2ba0001374e.csv.zst')

46.4 ms ± 454 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
%%timeit
df_arrow = io.load_path('/data/mlhd-zstd/d3/d38db397-d7b6-47fd-b57b-d2ba0001374e.csv.zst')

24.7 ms ± 804 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%%timeit
df_load_path_file = io.load_path_file_pandas('/home/snaek/MLHD/warehouse/samples/random_file_paths.txt')

3.32 s ± 44.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
df_load_path_file_arrow = io.load_path_file('/home/snaek/MLHD/warehouse/samples/random_file_paths.txt')

2.23 s ± 27.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
